In [ ]:
let input = System.IO.File.ReadAllText("day5_input.txt")

In [ ]:
let split (separator:string) (string:string) : list<string> = string.Split separator |> Array.toList

let parseStacks (stack_input:string) =
    stack_input
    |> split "\n"
    |> (fun list -> List.take (list.Length - 1) list)
    |> List.map (fun row ->
        row
        |> Seq.chunkBySize 4
        |> Seq.map (Seq.item 1)
        |> List.ofSeq
    )
    |> List.transpose
    |> List.map (List.except " ")

let parseInstructionLine (line:string) = 
    match split " " line with
    | [_; count; _; src; _; dest] -> (int count, (int src - 1), (int dest - 1))
    | _ -> failwith "Invalid instruction"

let parseInstructions (instruction_input:string) =
    instruction_input
    |> split "\n"
    |> Seq.map parseInstructionLine

let parseInput input =
    match split "\n\n" input with
    | [stacks; instructions] -> (parseStacks stacks, parseInstructions instructions)
    | _ -> failwith "Fail parseInput"

let popMany count stack =
    match count with
    | 0 -> ([], stack)
    | n -> 
        let rec popManyRec count stack acc =
            match count with
            | 0 -> acc
            | n when n > 0 ->
                let head::tail = stack
                popManyRec (n-1) tail (head::(fst acc), tail)
        popManyRec n stack ([], List.empty)

let apply stack (count, src, dest) = 
    let (move, new_src) = popMany count (List.item src stack)
    let new_dest = (List.rev move) @ (List.item dest stack)
    
    stack
    |> List.updateAt src new_src
    |> List.updateAt dest new_dest


In [ ]:
let (stacks, instructions) = parseInput input

instructions
|> Seq.fold apply stacks
|> Seq.map (Seq.item 0)
|> String.Concat